En este código se quiere realizar una implementación de un simulador de físicas sencillo.

In [ ]:
from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *
import sys
import numpy as np
import time as t

#Window size
width = 800
height = 800

xmin=-float(width/2)
xmax=float(width/2)
ymin=-float(height/2)
ymax=float(height/2)

objects = []

# PHYSICS CONSTANTS
radius = 20.0 #pix
g = np.array([0.0,-980])
BOUNCE_CONST = 0.7
SLIDE_CONST = 0.999

VEL_MAX = 100000

begin = t.time_ns()


class Object:

    def __init__(self, x, y):
        self.velocity = np.array([0,0])
        self.position = np.array([x,y])
        
    def update_physics(self,time_elapsed):
        
        new_v = np.add(self.velocity,np.multiply(g,time_elapsed)) #updates vel -> v = vo + at
        if not is_over_limit(new_v): #limit
                self.set_velocity(new_v)
        
        prev_pos = self.position
        new_pos = np.add(self.position,np.multiply(self.velocity,time_elapsed)) #update pos -> x = xo + vt
        self.set_position(new_pos)
            
        check_collisions(self)
        
        out_screen(self,new_pos)
        
        
        
    def set_velocity(self,new_vel):
        self.velocity = new_vel
    
    def set_position(self,new_pos):
        self.position = new_pos
        
    def get_vel_norm(self):
        return np.sqrt(self.velocity[0]**2 + self.velocity[1]**2)
    
    def to_string(self):
        return f"{self.position}-{self.velocity}"



def is_over_limit(v):
    return (np.sqrt(np.power(v[0],2) + np.power(v[1],2))) > VEL_MAX
            
# Process the new position on edges
def out_screen(object,new_pos):
    
    # LEFT AND RIGHT
    if new_pos[0] >= (xmax - radius) or new_pos[0] <= (xmin + radius):
        object.set_velocity(np.multiply(object.velocity,[-BOUNCE_CONST,SLIDE_CONST]))
        if new_pos[0] >= (xmax - radius): # RIGHT BORDER
            object.set_position(np.add(new_pos,[(xmax - radius)-new_pos[0],0])) 
          
        if new_pos[0] <= (xmin - radius): # LEFT
            object.set_position(np.add(new_pos,[(xmin + radius)-new_pos[0],0]))
        
    if new_pos[1] >= (ymax - radius) or new_pos[1] < (ymin + radius):
        object.set_velocity(np.multiply(object.velocity,[SLIDE_CONST,-BOUNCE_CONST]))
        
        #if new_pos[1] > (ymax - radius): # ARRIBA
         #  new_pos = np.add(new_pos,[0,(ymax-radius)-new_pos[1]])          
            
        if new_pos[1] < (ymin + radius): # ABAJO
            object.set_position(np.add(new_pos,[0,(ymin+radius)-new_pos[1]]))

def check_collisions(obj):
    global objects
    
    for other in objects:
        #diff vect calc
        diff_vec = np.subtract(other.position,obj.position)
        norm_diff_vec = np.sqrt(diff_vec[0]**2 + diff_vec[1]**2)
        
        if norm_diff_vec <= 2*radius and other != obj:
            
            #Velocity calcs for volume collisions
            ratio_o1 = obj.get_vel_norm()/norm_diff_vec
            ratio_o2 = other.get_vel_norm()/norm_diff_vec
            inverse_diff_vec = np.multiply(diff_vec,-1)
            
            # Check angle
            if det(diff_vec,obj.velocity)>0:
                new_vec_obj = np.multiply(inverse_diff_vec,ratio_o2)
                new_vec_other = np.multiply(diff_vec,ratio_o1)
            else:
                new_vec_obj = np.multiply(diff_vec,ratio_o2)
                new_vec_other = np.multiply(inverse_diff_vec,ratio_o1)
            
            #UPDATES VEL
            obj.set_velocity(new_vec_obj)
            other.set_velocity(new_vec_other)
            
            #UPDATES POS
            dim_overlap = radius - norm_diff_vec/2
            obj.set_position(np.add(obj.position,dim_overlap))
            other.set_position(np.subtract(other.position,dim_overlap))
            
            
def det(v1,v2):
    return v1[0]*v2[1]-v2[0]*v1[1]
    
# OPENGL FUNC

def init():
    glClearColor(0.0, 0.0, 0.0, 0.0)
    glMatrixMode(GL_PROJECTION)
    gluOrtho2D(xmin, xmax, ymin, ymax)
    glMatrixMode(GL_MODELVIEW)


# updates all velocities and positions
#def update():
def update():
    global objects, begin

    for object in objects:
        object.update_physics((t.time_ns() - begin)/(10**9))
        
    glutPostRedisplay()
    begin = t.time_ns()
    
# on left click creates a ball, on right one deletes all
def mouse(button, state, x, y):
    global objects
    
    realPosMousex = ((xmax-xmin)/width*x+xmin) #convert x,y from window to system x,y
    realPosMousey = ((ymin-ymax)/height*y+ymax)
    
    if button == GLUT_LEFT_BUTTON and state==GLUT_DOWN:
        objects.append(Object(realPosMousex,realPosMousey))

    if button == GLUT_RIGHT_BUTTON and state==GLUT_DOWN:
        objects = []
        
def draw():
    
    #cleans
    glClearColor(0.0, 0.0, 0.0,1.0);
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT);
    
    glEnable(GL_POINT_SMOOTH)
    glPointSize(2*radius)
    #draws all
    for obj in objects:
        glBegin(GL_POINTS)
        glVertex2fv(obj.position)
        glEnd()
    
    glDisable(GL_POINT_SMOOTH)
    glFlush()

glutInit(sys.argv)

glutInitDisplayMode(GLUT_SINGLE | GLUT_RGB)
# Atributos de la ventana
glutInitWindowPosition(950,200)
glutInitWindowSize(width,height)
glutCreateWindow("Particulas")

init()

glutDisplayFunc(draw)
glutMouseFunc(mouse)
glutIdleFunc(update)
glutMainLoop()